In [148]:
import nibabel as nib
import nilearn as nil
import numpy as np
import pandas as pd
import os
import os.path
from nilearn import plotting, image
from nipype.interfaces.spm import TwoSampleTTestDesign, EstimateModel, EstimateContrast, Threshold
from nipype.pipeline.engine import Workflow, Node, MapNode
from src.utils.data import getDataPandas, getPandas, getConfig

In [149]:
data = getDataPandas()
model_config = getConfig('model')
data_config = getConfig('data')
train_inds = data_config['train_index']
test_inds = data_config['test_index']
data_train = data.iloc[train_inds].reset_index(drop=True)
data_test = data.iloc[test_inds].reset_index(drop=True)
data_full = data.iloc[train_inds + test_inds].reset_index(drop=True)

In [150]:
train_good = data_train[data_train['CAT']==1]
train_bad = data_train[data_train['CAT']==0]
#train_good = data_full[data_full['CAT']==1]
#train_bad = data_full[data_full['CAT']==0]
img_good = train_good['T1_SGM_PATH']
img_bad = train_bad['T1_SGM_PATH']
img_good = img_good.apply(os.path.abspath)
img_bad = img_bad.apply(os.path.abspath)

In [151]:
age_good = train_good['AGE_AT_VISIT']
age_bad = train_bad['AGE_AT_VISIT']
sex_good = train_good['SEX']
sex_bad = train_bad['SEX']
tiv_good = train_good['TIV']
tiv_bad = train_bad['TIV']
agelist = age_good.tolist() + age_bad.tolist()
sexlist = sex_good.tolist() + sex_bad.tolist()
tivlist = tiv_good.tolist() + tiv_bad.tolist()

In [157]:
wf = Workflow(name='voxelwise', base_dir='tmp')

ttest = Node(TwoSampleTTestDesign(), name='ttest')
ttest.inputs.group1_files = img_good.tolist()
ttest.inputs.group2_files = img_bad.tolist()
ttest.inputs.covariates = [
    {'name': 'age', 'vector': agelist},
    {'name': 'sex', 'vector': sexlist},
    {'name': 'tiv', 'vector': tivlist}
]

model = Node(EstimateModel(), name='model')
model.inputs.estimation_method = {'Classical': 1}

contrast = Node(EstimateContrast(), name='contrast')
con1 = ('good>bad','T', ['Group_{1}','Group_{2}'],[1,-1])
con2 = ('bad>good','T', ['Group_{1}','Group_{2}'],[-1,1])
con_list = [con1, con2]
contrast.inputs.contrasts = con_list
contrast.inputs.group_contrast = True

threshold = MapNode(Threshold(
    contrast_index=1,
    use_topo_fdr=True,
    use_fwe_correction=False,
    extent_threshold=0,
    height_threshold=0.05,
    height_threshold_type='p-value',
    extent_fdr_p_threshold=0.05), name='thres', iterfield=['stat_image'])

wf.connect([
    (ttest, model, [('spm_mat_file', 'spm_mat_file')]),
    (model, contrast, [('spm_mat_file', 'spm_mat_file'), ('beta_images', 'beta_images'), ('residual_image', 'residual_image')]),
    (contrast, threshold, [('spm_mat_file', 'spm_mat_file'), ('spmT_images', 'stat_image')])
])

wf.run()

230324-17:15:22,586 nipype.workflow INFO:
	 Workflow voxelwise settings: ['check', 'execution', 'logging', 'monitoring']
230324-17:15:22,593 nipype.workflow INFO:
	 Running serially.
230324-17:15:22,594 nipype.workflow INFO:
	 [Node] Setting-up "voxelwise.ttest" in "/home/biobot/levodopa/levodopa/tmp/voxelwise/ttest".
230324-17:15:22,598 nipype.workflow INFO:
	 [Node] Cached "voxelwise.ttest" - collecting precomputed outputs
230324-17:15:22,599 nipype.workflow INFO:
	 [Node] "voxelwise.ttest" found cached.
230324-17:15:22,599 nipype.workflow INFO:
	 [Node] Setting-up "voxelwise.model" in "/home/biobot/levodopa/levodopa/tmp/voxelwise/model".
230324-17:15:22,603 nipype.workflow INFO:
	 [Node] Cached "voxelwise.model" - collecting precomputed outputs
230324-17:15:22,603 nipype.workflow INFO:
	 [Node] "voxelwise.model" found cached.
230324-17:15:22,604 nipype.workflow INFO:
	 [Node] Setting-up "voxelwise.contrast" in "/home/biobot/levodopa/levodopa/tmp/voxelwise/contrast".
230324-17:15:22,